# HW1 - Intro to the Map Reduce Paradigm  

__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Summer 2023`__

Welcome to Machine Learning at Scale! This first homework assignment introduces one of the core strategies in distributed processing: divide and conquer. We'll use the simplest of tasks, word counting, to illustrate the difference between a scalable and non-scalable algorithm. You will be working with the text of _Alice in Wonderland_ to put these ideas into practice using Python and Bash scripting. By the end of this week you should be able to:
* ... __describe__ the Bias-Variance tradeoff as it applies to Machine Learning.
* ... __explain__ why we consider word counting to be an "Embarrassingly Parallel" task.
* ... __estimate__ the runtime of embarrassingly parallel tasks using "back of the envelope" calculations.
* ... __implement__ a Map Reduce algorithm using the Command Line.
* ... __set-up__ a Docker container and know why we use them for this course.

You will also  become familiar (if you aren't already) with `defaultdict`, `re` and `time` in Python, linux piping and sorting, and Jupyter magic commands `%%writefile` and `%%timeit`. 


__IMPORTANT:__ If you're not familiar with linux, you should read the following tutorial reagrding **piping** and **redirecting**: https://ryanstutorials.net/linuxtutorial/piping.php You will need to understand the differences to answer some of the later questions.

__Please refer to the HW1 Assignment in Canvas for detailed submission instructions__.


# Notebook Set-Up
Before starting your homework run the following cells to confirm your setup.

In [2]:
# Confirm you are running Python 3:
import sys
sys.version_info

sys.version_info(major=3, minor=8, micro=15, releaselevel='final', serial=0)

In [3]:
# Imports:
import re
import sys

Create a folder for any data you download locally.

In [4]:
!mkdir data
# NOTE: the contents of this directory will be ignored by git.

# Question 1: Introductions

`The Caterpillar and Alice looked at each other for some time in silence: at last the Caterpillar took the hookah out of its mouth, and addressed her in a languid, sleepy voice. "Who are you?" said the Caterpillar.`

<div style="text-align: right"> -- Lewis Carroll, <i>Alice's Adventures in Wonderland</i>, Chapter 4 </div>


__a) Short Essay on Digital Campus:__ Tell us about yourself! Briefly describe where you live, how far along you are in MIDS, what other classes you are taking and what you want to get out of w261.

> __a)__ Type your answer here!

# Question 2: Bias - Variance
__a) Short Essay on Digital Campus:__ In 1-2 sentences (~200 and absolutely no more than 300 words!), explain the bias-variance trade off. Describe what it means to "decompose" sources of error. How is this used in machine learning? Please use mathematical equation(s) to support your explanation. (Use `$` signs to take advantage of $\LaTeX$ formatting, eg. `$f(x)$` will look like: $f(x)$). Please also cite any sources that informed your answer.

> __a)__ The bias-variance trade-off in machine learning refers to the balance between a model's simplicity (high bias, low variance) and its capacity to fit data noise (low bias, high variance). When decomposing total prediction error, it can be expressed as $\text{Error} = \text{Bias}^2 + \text{Variance} + \text{Irreducible Error}$, where Bias is the error from erroneous assumptions, Variance is the error from model's sensitivity to small fluctuations, and Irreducible Error is the noise in the data. In machine learning, understanding this trade-off aids in model selection and regularization. Overly complex models may have low bias but high variance, leading to overfitting; whereas too simple models may have high bias but low variance, leading to underfitting. Adjusting model complexity helps achieve an optimal balance between these errors, enhancing generalization on unseen data. (Source: Elements of Statistical Learning by Hastie, Tibshirani, and Friedman.)

# Question 3: Tokenizing
A number of our assignments this term will involve extracting information from text. A common preprocessing step when working with raw files is to 'tokenize' (i.e. extract words from) the text. Within the field of Natural Language Processing a lot of thought goes into what specific tokenizing makes most sense for a given task. For example, you might choose to remove punctuation or to consider punctuation symbols  'tokens' in their own right. __In this question you'll use the Python `re` module to create a tokenizer to use when you perform Word Count on the _Alice In Wonderland_ text.__

### Q3 Tasks:

* __a) Short Essay on Digital Campus:__ In the Naive Bayes algorithm (which we'll implement next week), we'll estimate the _likelihood_ of a word by counting the number of times it appears and dividing by the size of the vocabulary (total number of unique words). Using the text: *"Alice had an adventure that took alice to wonderland"*, give a concrete example of how two different tokenizers could cause us to get two different results on this calculation. [`HINT`: _you should not need to read up on Naive Bayes to answer this question._]  
  

* __b) Multiple Choice on Digital Campus:__ When tokenizing in this assignment we'll remove punctuation and discard numerical digits by making everything lowercase and then capturing only consecutive letters a to z. Suppose __`tokenize(x)`__ is a Python function that performs the desired tokenization. What would __`tokenize("By-the-bye, what became of Alice's 12 hats?!")`__ output? ['by', 'the', 'bye', 'what', 'became', 'of', 'alice', 's', 'hats']


* __c) code:__  Fill in the regular expression pattern in the cell labeled `part c` so that the subsequent call to `re.findall(RE_PATTERN, ...)` returns the tokenization described above. [`HINT`: _we've taken care of the lowercase part for you. If regex is new to you, check out the [`re`  documentation](https://docs.python.org/3/library/re.html) or [this PyMOTW tutorial](https://pymotw.com/2/re/)._]

### Q3 Student Answers:

> __a)__ Using a case sensitive tokenizer, the provided sentence would utilize the following tokens: ["Alice", "had", "an", "adventure", "that", "took", "alice", "to", "wonderland"], where the likelyhood of the word "alice" is 1/9. However, if we were to utilize a stop word tokenizer so that basic words like "the", "an", and "to" were removed from the text string, then we would end up with the following tokens:  ["alice", "had", "adventure", "that", "took", "wonderland"] and "alice" would have a likelyhood of 1/6. This shows how choosing different tokenizers drasitcally changes likelihood of words in a sentence. 

In [7]:
# Part C - Fill in the regular expression
RE_PATTERN = re.compile("[a-z]+")          

In [8]:
# Tokenize - DO NOT MODIFY THIS CELL, just run it as is to check your pattern
words = re.findall(RE_PATTERN, "By-the-bye, what became of Alice's 12 hats?!".lower())
print(words)

['by', 'the', 'bye', 'what', 'became', 'of', 'alice', 's', 'hats']


# Load the Data
`"Please would you tell me", said Alice, a little timidly, for she was not quite sure whether it was good manners for her to speak first, "why your cat grins like that?"`  
<div style="text-align: right">  -- Lewis Carroll, <i>Alice's Adventures in Wonderland</i>, Chapter 4</div>

For the main part of this assignment we'll be working with the free plain text version of _Alice's Adventures in Wonderland_ available from Project Gutenberg. __Use the first two cells below to download this text and preview the first few lines.__ 

In [9]:
# Download Full text 
# NOTE: feel free to use 'curl', 'wget' or equivalent command of your choice.
# !curl "http://www.gutenberg.org/files/11/11-0.txt" -o data/alice.txt
!wget -O data/alice.txt "http://www.gutenberg.org/files/11/11-0.txt"

--2023-09-04 17:25:44--  http://www.gutenberg.org/files/11/11-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/files/11/11-0.txt [following]
--2023-09-04 17:25:44--  https://www.gutenberg.org/files/11/11-0.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174313 (170K) [text/plain]
Saving to: ‘data/alice.txt’

data/alice.txt      100%[===================>] 170.23K  --.-KB/s    in 0.1s    

2023-09-04 17:25:51 (1.15 MB/s) - ‘data/alice.txt’ saved [174313/174313]



In [10]:
# Take a peak at the first few lines:

# NOTE: If you are working in JupyterLab on Docker you may not see the output 
# below due to an encoding issue... in that case, use a terminal on Docker to 
# execute this head command and confirm that the file has downloaded properly, 
# this encoding issue should not affect your work on subsequent HW items.

!head -n 6 data/alice.txt

﻿The Project Gutenberg eBook of Alice’s Adventures in Wonderland, by Lewis Carroll

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this eBook or online at


We'd also like you to develop a habit of creating small files with simulated data for use in developing, debugging, and testing your code. The Jupyter magic command `%%writefile` is a convenient way to do this. __Run the following cells to create a test data file for use in our word counting task.__

In [11]:
%%writefile data/alice_test.txt
This is a small test file. This file is for a test.
This small test file has two small lines.

Writing data/alice_test.txt


In [12]:
# confirm the file was created in the data directory using a grep command:
!ls data | grep test

alice_test.txt


# Question 4: Word Count in Python

Over the course of this term you will also become very familiar with writing Python programs that read from standard input and using Linux piping commands to run these programs and save their output to file. __In this question you will write a short Python script to perform the Word Count task and then run your script on the _Alice in Wonderland_ text__. You can think of this like a "baseline implementation" that we'll later compare to the parallelized version of the same task.

### Q4 Tasks:

* __a) Code in `wordCount.py` and Submit on Digital Campus:__ Complete the Python script in the file __`wordCount.py`__. Read the docstrings carefully to be sure you understand the expected behavior of this function. Please do not code outside of the marked location.


* __b) testing:__ Run the cell marked `part b` to call your script on the test file we created above. Confirm that your script returns the correct counts for each word by visually comparing the output to the test file. 


* __c) results:__ When you are confident in your implementation, run the cell marked `part c` to count the number of occurrences of each word in _Alice's Adventures in Wonderland_. In the same cell we'll pipe the output to file. Then use the provided `grep` commands to check your answers.


* __d) Short Essay on Digital Campus:__ Suppose you decide that you'd really like  a word and its plural (e.g. 'hatter' and 'hatters' or 'person' and 'people') to be counted as the same word. After we have run the wordcount would it be more efficient to post-process your output file or discard your output file and start the analysis over with a new tokenizer? Explain your reasoning briefly. 

### Q4 Student Answers:
> __a-c)__ _Complete the coding portions of this question before answering 'd'._

> __d)__ Type your answer here!

In [27]:
# part a - DO YOUR WORK IN wordCount.py
!cd media/notebooks/Assignments/HW1_cta && ls

README.md	       demo			    pWordCount_v2.sh
aggregateCounts_v1.py  hw1_Workbook.ipynb	    wordCount.py
aggregateCounts_v2.py  hw1_Workbook_dataproc.ipynb
data		       pWordCount_v1.sh


In [29]:
# part b - DO NOT MODIFY THIS CELL, just run it as is to test your script
!python media/notebooks/Assignments/HW1_cta/wordCount.py < data/alice_test.txt

this	3
is	2
a	2
small	3
test	3
file	3
for	1
has	1
two	1
lines	1


In [31]:
# part c - DO NOT MODIFY THIS CELL, just run it as is to perform the word count.
!python media/notebooks/Assignments/HW1_cta/wordCount.py < data/alice.txt > data/alice_counts.txt

Take a look at the first 10 words & their counts.

In [32]:
!head data/alice_counts.txt

the	1839
project	88
gutenberg	98
ebook	13
of	638
alice	403
s	222
adventures	11
in	435
wonderland	7


__Check your results:__ How many times does the word "alice" appear in the book? 

In [33]:
# EXPECTED OUTPUT: 403
!grep alice data/alice_counts.txt

alice	403


__Check your results:__ How many times does the word "hatter" appear in the book? 

In [34]:
# EXPECTED OUTPUT: 56
!grep hatter data/alice_counts.txt

hatter	56
hatters	1


__Check your results:__ How many times does the word "queen" appear in the book? 

In [35]:
# EXPECTED OUTPUT: 76
!grep queen data/alice_counts.txt

queen	76
queens	1


# Question 5: Unix Sorting Practice

Another common task in this course's assignments will be to make strategic use of sorting.     

### Q5 Tasks:
* __a) Multiple Choice on Digital Campus:__ What is the Big O complexity of the fastest comparison based sorting algorithms? [*`HINT`: If you need a Big O notation refresher, here's a [blog post](https://rob-bell.net/2009/06/a-beginners-guide-to-big-o-notation/), a [cheatsheet](http://bigocheatsheet.com), and a [thorough explanation](http://pages.cs.wisc.edu/~vernon/cs367/notes/3.COMPLEXITY.html).*] O($n \log n$)

* __b) Multiple Choice on Digital Campus:__ What is the default sorting algorithm in MapReduce? What is the Big O complexity of this algorithm? Why do you think this algorithm was chosen? [*`HINT`: Julius Ceasar! (week 1 slides)*] merge-sort

* __c) Code in notebook and Submit on Digital Campus:__ Write a unix command to check how many records are in your word count file.

* __d) Code in notebook and Submit on Digital Campus:__ Write a unix command to sort your word count file alphabetically. Save (i.e. [redirect](https://superuser.com/questions/277324/pipes-vs-redirects)) the results to `data/alice_counts_A-Z.txt`. [*`HINT`: if Unix sort commands are new to you, start with [this biowize blogpost](https://biowize.wordpress.com/2015/03/13/unix-sort-sorting-with-both-numeric-and-non-numeric-keys/) or [this unixschool tutorial](http://www.theunixschool.com/2012/08/linux-sort-command-examples.html)*]

* __e) Code in notebook and Submit on Digital Campus:__ Write a unix command to sort your word count file from highest to lowest count. Save (i.e. [redirect](https://superuser.com/questions/277324/pipes-vs-redirects)) your results to `data/alice_counts_sorted.txt`; then run the provided cell to print the top ten words. Compare your output to the expected output we provide.

### Q5 Student Answers:
> __a)__ Type your answer here!

> __b)__ Type your answer here!


In [36]:
# part c - write a unix command to check how many records are in your word count file
!wc -l data/alice_counts.txt

3006 data/alice_counts.txt


In [37]:
# part d - unix command to sort your word counts alphabetically 
!sort data/alice_counts.txt > data/alice_counts_A-Z.txt

In [38]:
# part d - DO NOT MODIFY THIS CELL, run it as is to confirm your sort worked
!head data/alice_counts_A-Z.txt   

a	695
abide	2
able	1
about	102
above	3
absence	1
absurd	2
accept	1
acceptance	1
accepted	2


In [46]:
# part e - unix command to sort your word counts from highest to lowest count
!sort -k2,2nr data/alice_counts.txt > data/alice_counts_sorted.txt

In [47]:
# part e - DO NOT MODIFY THIS CELL, run it as is to confirm your sort worked
!head data/alice_counts_sorted.txt

the	1839
and	942
to	811
a	695
of	638
it	610
she	553
i	546
you	486
said	462


<table>
<th>expected output for (d):</th>
<th>expected output for (e):</th>
<tr><td><pre>
a	695
abide	2
able	1
about	102
above	3
absence	1
absurd	2
accept	1
acceptance	1
accepted	2
</pre></td>
<td><pre>
the	1839
and	942
to	811
a	695
of	638
it	610
she	553
i	546
you	486
said	462
</pre></td></tr>
</table>

# Question 6: Parallel Word Count (part 1)
What would happen if we tried to run our script on a much larger dataset? For one thing, it would take longer to run. However there is a second concern. The Python object that aggregates our counts (`defaultdict`) exists in memory on the machine running this notebook. If the vocabulary is too large for the memory space available we would crash the notebook. The solution? Divide and Conquer! Instead of running the script on the whole dataset at once, we could split our text up in to smaller 'chunks' and process them independently of each other. __In this question you'll use a bash script to "parallelize" your Word Count.__


### Q6 Tasks:
* __a) Read provided code:__ The bash script `pWordCount_v1.sh` takes an input file, splits it into a specified number of 'chunks', and then applies an executable of your choice to each chunk. Read through this code and make sure you understand each line before you proceed. [*`HINT:` For now, ignore the 'student code' section -- you'll use that in part c.*]


* __b) Short Essay on Digital Campus:__ Below we've provided the command to use this script to apply your analysis (`wordCount.py`) to the _Alice_ text in 4 parallel processes. We'll redirect the results into a file called `alice_pCounts.txt.` Run this analysis and compare the count for the word 'alice' to your answer from Question 4. Explain what went wrong and describe what we have to add to `pWordCount_v1.sh` to fix the problem.


* __c) Code in notebook and Submit on Digital Campus:__ We've provided a python script, `aggregateCounts_v1.py`, which reads word counts from standard input and combines any duplicates it encounters. Read through this script to be sure you understand how it is written. Then follow the instructions in `pWordCount_v1.sh` to make a one-line modification so that it accepts `aggregateCounts_v1.py` as a 4th argument and uses this script to combine the chunk-ed word counts. Run the cell below to confirm that you now get the correct results for your 'alice' count.

### Q6 Student Answers:
> __b)__ Type your answer here!

In [49]:
# part b - make sure your scripts are executable (RUN THIS CELL AS IS)
!chmod a+x media/notebooks/Assignments/HW1_cta/pWordCount_v1.sh
!chmod a+x media/notebooks/Assignments/HW1_cta/wordCount.py

In [56]:
# part b - parallel word count on Alice text (RUN THIS CELL AS IS)
!media/notebooks/Assignments/HW1_cta/pWordCount_v1.sh 4 'data/alice.txt' 'wordCount.py' > 'data/alice_pCounts.txt'

README.md	       demo			    pWordCount_v2.sh
aggregateCounts_v1.py  hw1_Workbook.ipynb	    wordCount.py
aggregateCounts_v2.py  hw1_Workbook_dataproc.ipynb
data		       pWordCount_v1.sh
media/notebooks/Assignments/HW1_cta/pWordCount_v1.sh: line 69: ./wordCount.py: No such file or directory
media/notebooks/Assignments/HW1_cta/pWordCount_v1.sh: line 69: ./wordCount.py: No such file or directory
media/notebooks/Assignments/HW1_cta/pWordCount_v1.sh: line 69: ./wordCount.py: No such file or directory
media/notebooks/Assignments/HW1_cta/pWordCount_v1.sh: line 69: ./wordCount.py: No such file or directory


In [ ]:
# part b - check alice count (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

In [ ]:
# part c - make sure the aggregateCounts script is executable (RUN THIS CELL AS IS)
!chmod a+x aggregateCounts_v1.py

In [ ]:
# part c - parallel word count on Alice text (RUN THIS CELL AS IS)
!./pWordCount_v1.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v1.py' \
                   > 'data/alice_pCounts.txt'

In [ ]:
# part c - check alice count (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

# Question 7: Parallel Word Count (part 2)

Congratulations, you've just implemented a Map-Reduce algorithm! From here on out, we'll refer to the two Python scripts you passed to `pWordCount_v1.sh` as '_mapper_' and '_reducer_'. The bash script itself served as our '_framework_' -- it split up the original input, then ___mapped___ our word counting script on to each chunk, then ___aggregated (a.k.a. reduced)___ the resulting files by piping them into our collation script. Unfortunately, as you may have realized already, there is a major scalability concern with this particular implementation. __In this question you'll fix our implementation of parallel word count so that it will be scalable.__

__HINT:__ MapReduce uses the Merge-Sort algorithm under the hood. Linux `sort` command has a merge option which you can use to simulate the MapReduce framework. Use the `man sort` command to find more information on this option. 

### Q7 Tasks:

* __a) Multiple Choice on Digital Campus:__ What is the potential scalability problem with the provided implementation of `aggregateCounts_v1.py`? Why would this supposedly 'parallelized' Word Count potentially not work on a really large input corpus. [*`HINT:` See the intro to Q6*]


* __b) Code in `pWordCount_v2.sh` and Submit on Digital Campus:__ Fortunately, a 'strategic sort' can solve this problem. Read the instructions at the top of `pWordCount_v2.sh` carefully then make your changes that alphabetically sort the output from the mappers (`countfiles`) before piping them into the reducer script.


* __c) Code in `aggregateCounts_v2.py` and Submit on Digital Campus:__ Write the main part of `aggregateCounts_v2.py` so that it takes advantage of the sorted input to add duplicate counts without storing the whole vocabulary in memory. Refer to the file docstring for more detailed instructions. Confirm that your implementation works by running it on both the test and true data files.


* __d) Short Essay on Digital Campus:__ If you are paying close attention, this rewritten reducer sets us up for a truly scalable solution, but doesn't get us all the way there. In particular, while we chunked our data so it can be processed by multiple mappers, we're still streaming the entire dataset through one reduce script. If the vocabulary is too large to fit on a single computer, we might split the word counts in half after sorting them, then perform the reducing on two separate machines. Explain what could go wrong with this approach. (For now, ignore the question of how we'd sort a dataset that is too large to fit on a single machine and just focus on what might be wrong about the result of this split-in-half reducing).


* __e) Short Essay on Digital Campus:__ Can you come up with a different way of splitting up the data that would allow us to perform the reducing on separate machines without needing any postprocessing? This is a theoretical question -- don't worry if you don't know how to implement your idea in a bash script, just describe how you'd want to split the sorted counts into different files to be reduced separately.

### Q7 Student Answers:

> __a)__ Type your answer here!

> __b-c)__ _complete the coding portions of this question before answering d and e._

> __d)__ Type your answer here!

> __e)__ Type your answer here!

In [ ]:
# Add permissions to your new files (RUN THIS CELL AS IS)
!chmod a+x pWordCount_v2.sh
!chmod a+x aggregateCounts_v2.sh

In [ ]:
# part c - test your code on the test file (RUN THIS CELL AS IS)
!./pWordCount_v2.sh 4 'data/alice_test.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v2.py'

In [ ]:
# part c - run your code on the Alice file (RUN THIS CELL AS IS)
!./pWordCount_v2.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v2.py' \
                   > 'data/alice_pCounts.txt'

In [ ]:
# part c - confirm that your 'alice' count is correct (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

# Question 8: Scalability Considerations

In your reading for Week 2's live session, [Chapter1, section 2](https://lintool.github.io/MapReduceAlgorithms/MapReduce-book-final.pdf) of _Data Intensive Text Processing with MapReduce_, Lin and Dyer discuss a number of "Big Ideas" that underlie large scale processing: __scale "out," not "up"; assume failures are common; move processing to the data; process data sequentially and avoid random access; hide system-level details from the application developer; and seamless scalability.__ Part of our work this semester will be to interact with these ideas in a practical way, not just a conceptual one.

### Q8 Tasks:

* __a) Short Essay on Digital Campus:__ What do Lin and Dyer consider the two features of an "ideal algorithm" from a scalability perspective?


* __b) Multiple Choice on Digital Campus:__ The mapper script below (created on the fly using a little Jupyter magic) will help us illustrate the concept of scalability. Run the provided code which passes this mapper script to our parallel computation 'framework' and runs the 'analysis' on the _Alice_ text file. Note that we've omitted a reducer for simplicity. What do you observe about the time it takes to run this "algorithm" when we use 1, 2 and 4 partitions? Does it meet Lin and Dyer's criteria?

* __c) Short Essay on Digital Campus:__ Let's try something similar with your Word Count analysis. Run the provided code to time your implementation with 1, 2, 4 and 8 partitions. What do you observe about the runtimes? Does this match your expectation? Speculate about why we might be seeing these times. What conclusions should we draw about the scalability of our implementation? [*`HINT:` Consider the limitations of both your machine and our implementation... there are some competing forces at work, what are they?*]


* __d) Multiple Choice on Digital Campus:__ Which components of your Map-Reduce algorithm are affected by a change in the number of partitions? Does increasing the number of partitions increase or decrease the total time spent on each of these portions of the task? What tradeoff does this cause?

### Q8 Student Answers:

> __a)__ Type your answer here!

> __b)__ Type your answer here!

> __c)__ Type your answer here!

> __d)__ Type your answer here!

__Run the following cells to create the mapper referenced in `part b`__

In [ ]:
!mkdir demo

In [ ]:
%%writefile demo/mapper.py
#!/usr/bin/env python
"""
This mapper reads from STDIN and waits 0.001 seconds per line.
Its only purpose is to demonstrate one of the scalability ideas.
"""
import sys
import time
for line in sys.stdin:
    time.sleep(0.001)

In [ ]:
# Make sure the mapper is executable
!chmod a+x demo/mapper.py

__Run the next three cells to apply our demo mapper with 1, 2 and 4 partitions.__

In [ ]:
%%timeit
!./pWordCount_v2.sh 1 'data/alice.txt' 'demo/mapper.py'

In [ ]:
%%timeit
!./pWordCount_v2.sh 2 'data/alice.txt' 'demo/mapper.py'

In [ ]:
%%timeit
!./pWordCount_v2.sh 4 'data/alice.txt' 'demo/mapper.py'

__Run the following cells to repeat this process with your word count algorithm.__

In [ ]:
%%timeit
!./pWordCount_v2.sh 1 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

In [ ]:
%%timeit
!./pWordCount_v2.sh 2 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

In [ ]:
%%timeit
!./pWordCount_v2.sh 4 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

In [ ]:
%%timeit
!./pWordCount_v2.sh 8 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

In [ ]:
%%timeit
!./pWordCount_v2.sh 16 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

In [ ]:
%%timeit
!./pWordCount_v2.sh 32 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

# Question 9: Embarrassingly Parallel
`"If any one of them can explain it," said Alice, (she had grown so large in the last few minutes that she wasn’t a bit afraid of interrupting him,) "I’ll give him sixpence. I don’t believe there’s an atom of meaning in it."`
<div style="text-align: right">  -- Lewis Carroll, <i>Alice's Adventures in Wonderland</i>, Chapter 12</div>

### Q9 Tasks:

* __a) Short Essay on Digital Campus:__ Describe what we mean by 'Embarrassingly Parallel' in terms of word counting. Does this term describe a 'task'? An 'implementation of a task'? 

* __b) Short Essay on Digital Campus:__ Define this concept in terms of 'associative' and 'commutative' operations. [*`HINT:` Refer to Chapter 2 in DITP*]

### Q9 Student Answers:

> __a)__ Type your answer here!

> __b)__ Type your answer here!

### Congratulations, you have completed HW1! Please refer to the HW1 Assignment in Canvas for detailed submission instructions.